In [1]:
import numpy as np
np.set_printoptions(threshold=10000,suppress=True)
import pandas as pd
import warnings
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

In [2]:
# Import du jeu de données
dfCredit = pd.read_csv('credit.data', sep='\t')
nbColumns = len(dfCredit.columns)
print("nb of columns", nbColumns)
X = dfCredit.iloc[:,:nbColumns-1]
Y = dfCredit.iloc[:,nbColumns-1]

nb of columns 16


In [3]:
dfCredit.head()
#Y.head()

,b,30.83,0,u,g,w,v,1.25,t,t.1,1,f,g.1,202,0.1,+
0,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43,560,+
1,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,280,824,+
2,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100,3,+
3,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120,0,+
4,b,32.08,4.000,u,g,m,v,2.50,t,f,0,t,g,360,0,+


In [38]:
import re

def is_num(entry):
    _str = str(entry)
    #print("is_num", _str)
    regex_float = '[+-]?[0-9]+\.[0-9]+'
    regex_int = '[0-9]+'

    if(re.search(regex_int, _str)):  
        #print(str, "Integer number")  
        return True
    elif(re.search(regex_float, _str)):  
        #print(str, "Floating point number")  
        return True
    else:  
        #print(_str, "not a number")  
        return False
        
print(is_num('013'))

# Transformation en array
arrayCredit = dfCredit.to_numpy()
# Extraction des variables caractéristiques
arrayX = (dfCredit.iloc[:,:nbColumns-1]).to_numpy()
# Extraction de la varaiable à prédire
arrayY = (dfCredit.iloc[:,nbColumns-1]).to_numpy()
print("arrayX", arrayX.shape)
#print(arrayCredit, arrayX, arrayY)

# Sélection des colonnes de type numérique ()
row1 = arrayX[1,:]
print(row1, type(row1[1]))
numeric_columns=[]
numeric_columns = [is_num(row1[i]  )  for i in range(len(row1))]

    
print("numeric_columns", numeric_columns)
#selections = np.array([True, False, True])
arrayX_num = arrayX[:, numeric_columns]


arrayY_num = np.zeros(len(arrayY));
# Conversion de tous les éléments du tableau en float
print("arrayY", arrayY[0],arrayY[1] )
lengthY = len(arrayY)
for idx in range(lengthY):
    #print("step1",idx,arrayY[idx])
    if(arrayY[idx] == '+'):
        #print('set 1')
        arrayY_num[idx] = 1
#print("y value",idx_row, arrayY[idx_row])
#print("arrayY_num", arrayY_num)



idx_col = 0
for row in arrayX_num:    
    idx_row = 0
    for item in row:
        if(isinstance(item, str)):
            if("?"==item):
                item = float('nan')
            arrayX_num[idx_col][idx_row] = float(item)
        idx_row+=1
        #print(item) 
    idx_col+=1
   


def has_nan(row):
    array_sum = np.sum(row)
    return np.isnan(array_sum)
    
# Suppression des lignes comporanant un nan
print( "before NaN remove arrayX_num", arrayX_num.shape, "arrayY_num", arrayY_num.shape) 

selected_rows = []
arrayX_clean = []
print("test1", arrayX_num[1,:])
fNaN = float('nan')
for row in arrayX_num:
    if has_nan(row):
        #print("nan found in ", row)
        selected_rows.append(False)
    else:
        selected_rows.append(True)
        #arrayX_clean.append(row)

arrayX_num = arrayX_num[selected_rows,:]
arrayY_num = arrayY_num[selected_rows]

print("after NaN remove arrayX_num",arrayX_num.shape, "arrayY_num", arrayY_num.shape) 


True
arrayX (687, 15)
['a' '24.50' 0.5 'u' 'g' 'q' 'h' 1.5 't' 'f' 0 'f' 'g' '280' 824] <class 'str'>
numeric_columns [False, True, True, False, False, False, False, True, False, False, True, False, False, True, True]
arrayY + +
before NaN remove (687, 6)
test1 [24.5 0.5 1.5 0 280.0 824]
after NaN remove arrayX_num (665, 6) arrayY_num (665,)


In [5]:
MP = 100*np.sum(Y == 0)/len(Y)
BP = 100*np.sum(Y == 1)/len(Y)

# Apprentissage

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

In [7]:
#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.5, random_state=1)
#print('train', X_train.shape, y_train.shape, 'test', X_test.shape, y_test.shape)


train (343, 15) (343,) test (344, 15) (344,)


In [8]:

""" 
    Fontion utilitaire d'affichage des résultat d'un algorithme : matrice de confusion, accuracy, precision
"""
def log_result(Y_test, Y_pred, algoType, note):
    print("--- ", algoType, " ---" )
    m_confusion = confusion_matrix(Y_test, Y_pred )
    print("Matrice de confusion", algoType, note)
    print(m_confusion)
    acc = round(100*accuracy_score(Y_test, Y_pred), 2)
    prec = round(100*precision_score(Y_test, Y_pred), 2)
    print(algoType, note, ' : Accuracy  ', acc , 'Precision' , prec)

In [9]:
""" Fonction qui applique successivement les 3 algorithmes : 
        . Arbre de décision
        . K plus proches voisins
        . Perceptron multi-couches
"""
def apply_classifier(X_train, _train, X_test, Y_test, note):
    # Arbre de décision
    DT = DecisionTreeClassifier(criterion = 'gini', random_state=1)
    DT.fit(X_train, y_train)
    Y_pred = DT.predict(X_test)
    log_result(Y_test, Y_pred, 'arbre de décision', note)
    
    # K plus prohces voisins
    KNN = KNeighborsClassifier(n_neighbors=5)
    KNN.fit(X_train, y_train)
    Y_pred = KNN.predict(X_test)
    log_result(Y_test, Y_pred, 'K plus proches voisins', note)

    #MultilayerPerceptron à deux couches de tailles respectives 40 et 20 et random_state=1
    MLP = MLPClassifier( alpha=1e-5, hidden_layer_sizes=(40, 20), random_state=1)
    MLP.fit(X_train, y_train)
    Y_pred = MLP.predict(X_test)
    log_result(Y_test, Y_pred, 'Perceptron multi couches', note)
    

# bababa

# bababa2